# Sora with Azure AI Foundry

Sora is an AI model from OpenAI that can create realistic and imaginative video scenes from text instructions. The model is capable of generating a wide range of video content, including realistic scenes, animations, and special effects. Several video resolutions and durations are supported.

https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/video-generation

In [1]:
import base64
import datetime
import os
import requests
import sys
import time

from dotenv import load_dotenv
from IPython.display import Video, FileLink
from moviepy import *

In [2]:
sys.version

'3.10.14 (main, May  6 2024, 19:42:50) [GCC 11.2.0]'

In [3]:
print(f"Today is {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today is 02-Jun-2025 10:03:43


## Settings

In [4]:
OUTPUT_DIR = "videos"

os.makedirs(OUTPUT_DIR, exist_ok=True)

In [5]:
load_dotenv("azure.env")

endpoint = os.environ['AZURE_OPENAI_ENDPOINT']
api_key = os.environ['AZURE_OPENAI_API_KEY']

model = "sora"

# Function

In [6]:
def sora(prompt, width=480, height=480, n_seconds=5):
    """
    Generates a video based on the given prompt using the SORA model.

    Parameters:
    prompt (str): The text prompt to generate the video.
    width (int): The width of the video. Supported values are 480, 854, 720, 1080, and 1920.
    height (int): The height of the video. Supported values are 480, 854, 720, 1080, and 1920.
    n_seconds (int): The duration of the video in seconds. Must be between 1 and 20 seconds.
    n_variants (int): The number of video variants to generate.
    
    Returns:
    str: The filename of the generated video.

    Raises:
    Exception: If the video generation job fails or no generations are found.
    """
    start = time.time()

    api_version = 'preview'
    headers = {"api-key": api_key, "Content-Type": "application/json"}

    idx = datetime.datetime.today().strftime('%d%b%Y_%H%M%S')
    suffix = prompt[:30].replace(",", "_").replace(".", "_").replace(" ", "_")
    output_filename = os.path.join(OUTPUT_DIR, f"sora_{idx}_{suffix}.mp4")

    # 1. Create a video generation job
    create_url = f"{endpoint}/openai/v1/video/generations/jobs?api-version={api_version}"
    
    body = {
        "prompt": prompt,
        "width": width,  # 480x480, 480x854, 854x480, 720x720, 720x1280, 1280x720, 1080x1080, 1080x1920, 1920x1080.
        "height": height,  # 480x480, 480x854, 854x480, 720x720, 720x1280, 1280x720, 1080x1080, 1080x1920, 1920x1080.
        "n_seconds": n_seconds,  # between 1 and 20 seconds
        "model": model,  # SORA model
    }
    response = requests.post(create_url, headers=headers, json=body)
    response.raise_for_status()

    now = datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')
    print(f"{now} Full response JSON:", response.json())
    print()

    job_id = response.json()["id"]
    now = datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')
    print(f"{now} Job created: {job_id}")

    # 2. Poll for job status
    status_url = f"{endpoint}/openai/v1/video/generations/jobs/{job_id}?api-version={api_version}"
    status = None

    while status not in ("succeeded", "failed", "cancelled"):
        time.sleep(5)  # Wait before polling again
        status_response = requests.get(status_url, headers=headers).json()
        status = status_response.get("status")
        now = datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')
        print(f"{now} Job status: {status}")

    # 3. Retrieve generated video
    if status == "succeeded":
        generations = status_response.get("generations", [])

        if generations:
            now = datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')
            print(f"\n{now} ✅ Done. Video generation succeeded.")
            generation_id = generations[0].get("id")
            video_url = f"{endpoint}/openai/v1/video/generations/{generation_id}/content/video?api-version={api_version}"
            video_response = requests.get(video_url, headers=headers)

            if video_response.ok:
                # Downloading the video
                print("\nDownloading the video...")
                with open(output_filename, "wb") as file:
                    file.write(video_response.content)
                    print(f"SORA Generated video saved: '{output_filename}'")

                elapsed = time.time() - start
                minutes, seconds = divmod(elapsed, 60)
                print(f"Done in {minutes:.0f} minutes and {seconds:.0f} seconds")

                return output_filename
        else:
            raise Exception("Error. No generations found in job result.")
    else:
        raise Exception(f"Error. Job did not succeed. Status: {status}")

## Example

In [7]:
prompt = "A young boy and his father playing together in the ocean on the beach."

generated_video = sora(prompt, width=480, height=480, n_seconds=5)

02-Jun-2025 10:03:44 Full response JSON: {'object': 'video.generation.job', 'id': 'task_01jwr1hp79ftw8ah1stqvdzk4r', 'status': 'queued', 'created_at': 1748858624, 'finished_at': None, 'expires_at': None, 'generations': [], 'prompt': 'A young boy and his father playing together in the ocean on the beach.', 'model': 'sora', 'n_variants': 1, 'n_seconds': 5, 'height': 480, 'width': 480, 'failure_reason': None}

02-Jun-2025 10:03:44 Job created: task_01jwr1hp79ftw8ah1stqvdzk4r
02-Jun-2025 10:03:49 Job status: running
02-Jun-2025 10:03:55 Job status: running
02-Jun-2025 10:04:00 Job status: processing
02-Jun-2025 10:04:05 Job status: succeeded

02-Jun-2025 10:04:05 ✅ Done. Video generation succeeded.

SORA Generated video saved: 'videos/sora_02Jun2025_100343_A_young_boy_and_his_father_pla.mp4'
Done in 0 minutes and 25 seconds


In [8]:
Video(generated_video)

In [9]:
video_link = FileLink(path=generated_video)
video_link

/mnt/batch/tasks/shared/LS_root/mounts/clusters/seretkow8/code/Users/seretkow/SORA/videos/sora_02Jun2025_100343_A_young_boy_and_his_father_pla.mp4

## Another example

In [10]:
prompt = "A close up view of a glass sphere that has a zen garden within it. There is a small dwarf in the sphere who is raking the zen garden and creating patterns in the sand."

generated_video = sora(prompt, width=480, height=480, n_seconds=5)

02-Jun-2025 10:04:09 Full response JSON: {'object': 'video.generation.job', 'id': 'task_01jwr1jey8fmsarw05npkcvyx7', 'status': 'queued', 'created_at': 1748858649, 'finished_at': None, 'expires_at': None, 'generations': [], 'prompt': 'A close up view of a glass sphere that has a zen garden within it. There is a small dwarf in the sphere who is raking the zen garden and creating patterns in the sand.', 'model': 'sora', 'n_variants': 1, 'n_seconds': 5, 'height': 480, 'width': 480, 'failure_reason': None}

02-Jun-2025 10:04:09 Job created: task_01jwr1jey8fmsarw05npkcvyx7
02-Jun-2025 10:04:15 Job status: running
02-Jun-2025 10:04:20 Job status: processing
02-Jun-2025 10:04:25 Job status: succeeded

02-Jun-2025 10:04:25 ✅ Done. Video generation succeeded.

SORA Generated video saved: 'videos/sora_02Jun2025_100409_A_close_up_view_of_a_glass_sph.mp4'
Done in 0 minutes and 18 seconds


In [11]:
Video(generated_video)

In [12]:
video_link = FileLink(path=generated_video)
video_link

/mnt/batch/tasks/shared/LS_root/mounts/clusters/seretkow8/code/Users/seretkow/SORA/videos/sora_02Jun2025_100409_A_close_up_view_of_a_glass_sph.mp4

## Another example

In [13]:
prompt = "Several giant wooly mammoths approach treading through a snowy meadow, their long wooly fur lightly blows in the wind as they walk, snow covered trees and dramatic snow capped mountains in the distance, mid afternoon light with wispy clouds and a sun high in the distance creates a warm glow, the low camera view is stunning capturing the large furry mammal with beautiful photography, depth of field."

generated_video = sora(prompt,
                       width=1280,
                       height=720,
                       n_seconds=5,
                       )

02-Jun-2025 10:04:27 Full response JSON: {'object': 'video.generation.job', 'id': 'task_01jwr1k0veevyrdmkqh58enarb', 'status': 'queued', 'created_at': 1748858667, 'finished_at': None, 'expires_at': None, 'generations': [], 'prompt': 'Several giant wooly mammoths approach treading through a snowy meadow, their long wooly fur lightly blows in the wind as they walk, snow covered trees and dramatic snow capped mountains in the distance, mid afternoon light with wispy clouds and a sun high in the distance creates a warm glow, the low camera view is stunning capturing the large furry mammal with beautiful photography, depth of field.', 'model': 'sora', 'n_variants': 1, 'n_seconds': 5, 'height': 720, 'width': 1280, 'failure_reason': None}

02-Jun-2025 10:04:27 Job created: task_01jwr1k0veevyrdmkqh58enarb
02-Jun-2025 10:04:33 Job status: queued
02-Jun-2025 10:04:38 Job status: running
02-Jun-2025 10:04:44 Job status: running
02-Jun-2025 10:04:49 Job status: running
02-Jun-2025 10:04:54 Job sta

In [14]:
Video(generated_video, width=640)

In [15]:
video_link = FileLink(path=generated_video)
video_link

/mnt/batch/tasks/shared/LS_root/mounts/clusters/seretkow8/code/Users/seretkow/SORA/videos/sora_02Jun2025_100427_Several_giant_wooly_mammoths_a.mp4

## Another example

In [16]:
prompt = "Drone view of waves crashing against the rugged cliffs along Big Sur's garay point beach. The crashing blue waters create white-tipped waves, while the golden light of the setting sun illuminates the rocky shore. A small island with a lighthouse sits in the distance, and green shrubbery covers the cliff's edge. The steep drop from the road down to the beach is a dramatic feat, with the cliff's edges jutting out over the sea. This is a view that captures the raw beauty of the coast and the rugged landscape of the Pacific Coast Highway."

generated_video = sora(prompt,
                       width=1280,
                       height=720,
                       n_seconds=15,
                       )

02-Jun-2025 10:05:05 Full response JSON: {'object': 'video.generation.job', 'id': 'task_01jwr1m5v2ek284z52ybkb223s', 'status': 'queued', 'created_at': 1748858705, 'finished_at': None, 'expires_at': None, 'generations': [], 'prompt': "Drone view of waves crashing against the rugged cliffs along Big Sur's garay point beach. The crashing blue waters create white-tipped waves, while the golden light of the setting sun illuminates the rocky shore. A small island with a lighthouse sits in the distance, and green shrubbery covers the cliff's edge. The steep drop from the road down to the beach is a dramatic feat, with the cliff's edges jutting out over the sea. This is a view that captures the raw beauty of the coast and the rugged landscape of the Pacific Coast Highway.", 'model': 'sora', 'n_variants': 1, 'n_seconds': 15, 'height': 720, 'width': 1280, 'failure_reason': None}

02-Jun-2025 10:05:05 Job created: task_01jwr1m5v2ek284z52ybkb223s
02-Jun-2025 10:05:11 Job status: queued
02-Jun-2025

In [17]:
Video(generated_video)

In [18]:
video_link = FileLink(path=generated_video)
video_link

/mnt/batch/tasks/shared/LS_root/mounts/clusters/seretkow8/code/Users/seretkow/SORA/videos/sora_02Jun2025_100505_Drone_view_of_waves_crashing_a.mp4

## Another example

In [19]:
prompt = "Extreme close up of a 25 year old woman's eye blinking, cinematic film shot in 70mm, depth of field, vivid colors, cinematic"

generated_video = sora(prompt,
                       width=1280,
                       height=720,
                       n_seconds=5,
                       )

02-Jun-2025 10:07:11 Full response JSON: {'object': 'video.generation.job', 'id': 'task_01jwr1r0hdewvbkkvy2p5mwts7', 'status': 'queued', 'created_at': 1748858831, 'finished_at': None, 'expires_at': None, 'generations': [], 'prompt': "Extreme close up of a 25 year old woman's eye blinking, cinematic film shot in 70mm, depth of field, vivid colors, cinematic", 'model': 'sora', 'n_variants': 1, 'n_seconds': 5, 'height': 720, 'width': 1280, 'failure_reason': None}

02-Jun-2025 10:07:11 Job created: task_01jwr1r0hdewvbkkvy2p5mwts7
02-Jun-2025 10:07:16 Job status: preprocessing
02-Jun-2025 10:07:22 Job status: running
02-Jun-2025 10:07:27 Job status: running
02-Jun-2025 10:07:33 Job status: running
02-Jun-2025 10:07:38 Job status: processing
02-Jun-2025 10:07:43 Job status: succeeded

02-Jun-2025 10:07:43 ✅ Done. Video generation succeeded.

SORA Generated video saved: 'videos/sora_02Jun2025_100711_Extreme_close_up_of_a_25_year_.mp4'
Done in 0 minutes and 38 seconds


In [20]:
Video(generated_video, width=860)

In [21]:
video_link = FileLink(path=generated_video)
video_link

/mnt/batch/tasks/shared/LS_root/mounts/clusters/seretkow8/code/Users/seretkow/SORA/videos/sora_02Jun2025_100711_Extreme_close_up_of_a_25_year_.mp4

## Another example

In [22]:
prompt = "Extreme close up of a 50 year old male's eye blinking, cinematic film shot in 70mm, depth of field, vivid colors, cinematic"

generated_video = sora(prompt,
                       width=1280,
                       height=720,
                       n_seconds=5,
                       )

02-Jun-2025 10:07:49 Full response JSON: {'object': 'video.generation.job', 'id': 'task_01jwr1s5a9fgj8tvnfs4ncg513', 'status': 'queued', 'created_at': 1748858869, 'finished_at': None, 'expires_at': None, 'generations': [], 'prompt': "Extreme close up of a 50 year old male's eye blinking, cinematic film shot in 70mm, depth of field, vivid colors, cinematic", 'model': 'sora', 'n_variants': 1, 'n_seconds': 5, 'height': 720, 'width': 1280, 'failure_reason': None}

02-Jun-2025 10:07:49 Job created: task_01jwr1s5a9fgj8tvnfs4ncg513
02-Jun-2025 10:07:54 Job status: queued
02-Jun-2025 10:07:59 Job status: running
02-Jun-2025 10:08:05 Job status: running
02-Jun-2025 10:08:10 Job status: running
02-Jun-2025 10:08:16 Job status: processing
02-Jun-2025 10:08:21 Job status: succeeded

02-Jun-2025 10:08:21 ✅ Done. Video generation succeeded.

SORA Generated video saved: 'videos/sora_02Jun2025_100748_Extreme_close_up_of_a_50_year_.mp4'
Done in 0 minutes and 38 seconds


In [23]:
Video(generated_video, width=860)

In [24]:
video_link = FileLink(path=generated_video)
video_link

/mnt/batch/tasks/shared/LS_root/mounts/clusters/seretkow8/code/Users/seretkow/SORA/videos/sora_02Jun2025_100748_Extreme_close_up_of_a_50_year_.mp4

## Another example

In [25]:
prompt = "Reflections in the window of a train traveling through the Tokyo suburbs."

generated_video = sora(prompt,
                       width=1280,
                       height=720,
                       n_seconds=10,
                       )

02-Jun-2025 10:08:26 Full response JSON: {'object': 'video.generation.job', 'id': 'task_01jwr1ta4qfb18fx1he5xsns07', 'status': 'queued', 'created_at': 1748858906, 'finished_at': None, 'expires_at': None, 'generations': [], 'prompt': 'Reflections in the window of a train traveling through the Tokyo suburbs.', 'model': 'sora', 'n_variants': 1, 'n_seconds': 10, 'height': 720, 'width': 1280, 'failure_reason': None}

02-Jun-2025 10:08:26 Job created: task_01jwr1ta4qfb18fx1he5xsns07
02-Jun-2025 10:08:32 Job status: queued
02-Jun-2025 10:08:37 Job status: running
02-Jun-2025 10:08:43 Job status: running
02-Jun-2025 10:08:48 Job status: running
02-Jun-2025 10:08:53 Job status: running
02-Jun-2025 10:08:59 Job status: running
02-Jun-2025 10:09:04 Job status: running
02-Jun-2025 10:09:10 Job status: running
02-Jun-2025 10:09:15 Job status: running
02-Jun-2025 10:09:20 Job status: processing
02-Jun-2025 10:09:26 Job status: processing
02-Jun-2025 10:09:31 Job status: succeeded

02-Jun-2025 10:09:

In [26]:
Video(generated_video, width=860)

In [27]:
video_link = FileLink(path=generated_video)
video_link

/mnt/batch/tasks/shared/LS_root/mounts/clusters/seretkow8/code/Users/seretkow/SORA/videos/sora_02Jun2025_100826_Reflections_in_the_window_of_a.mp4

## Another example

In [28]:
prompt = "A drone view of flock of paper airplanes multicolors flutters through a dense jungle, weaving around trees as if they were migrating birds."

generated_video = sora(prompt,
                       width=1280,
                       height=720,
                       n_seconds=7,
                       )

02-Jun-2025 10:09:40 Full response JSON: {'object': 'video.generation.job', 'id': 'task_01jwr1wj27fke9jhgwhftvggzy', 'status': 'queued', 'created_at': 1748858980, 'finished_at': None, 'expires_at': None, 'generations': [], 'prompt': 'A drone view of flock of paper airplanes multicolors flutters through a dense jungle, weaving around trees as if they were migrating birds.', 'model': 'sora', 'n_variants': 1, 'n_seconds': 7, 'height': 720, 'width': 1280, 'failure_reason': None}

02-Jun-2025 10:09:40 Job created: task_01jwr1wj27fke9jhgwhftvggzy
02-Jun-2025 10:09:45 Job status: preprocessing
02-Jun-2025 10:09:51 Job status: running
02-Jun-2025 10:09:56 Job status: running
02-Jun-2025 10:10:02 Job status: running
02-Jun-2025 10:10:07 Job status: running
02-Jun-2025 10:10:12 Job status: running
02-Jun-2025 10:10:18 Job status: processing
02-Jun-2025 10:10:23 Job status: succeeded

02-Jun-2025 10:10:23 ✅ Done. Video generation succeeded.

SORA Generated video saved: 'videos/sora_02Jun2025_1009

In [29]:
Video(generated_video, width=860)

In [30]:
video_link = FileLink(path=generated_video)
video_link

/mnt/batch/tasks/shared/LS_root/mounts/clusters/seretkow8/code/Users/seretkow/SORA/videos/sora_02Jun2025_100940_A_drone_view_of_flock_of_paper.mp4

## Another exemple

In [31]:
prompt = "Street of Tokyo, rainy day with people walking in a street, restaurants in the street."

generated_video = sora(prompt,
                       width=1280,
                       height=720,
                       n_seconds=7,
                       )

02-Jun-2025 10:10:30 Full response JSON: {'object': 'video.generation.job', 'id': 'task_01jwr1y2q1fy1at3w70k52s6xd', 'status': 'queued', 'created_at': 1748859030, 'finished_at': None, 'expires_at': None, 'generations': [], 'prompt': 'Street of Tokyo, rainy day with people walking in a street, restaurants in the street.', 'model': 'sora', 'n_variants': 1, 'n_seconds': 7, 'height': 720, 'width': 1280, 'failure_reason': None}

02-Jun-2025 10:10:30 Job created: task_01jwr1y2q1fy1at3w70k52s6xd
02-Jun-2025 10:10:35 Job status: preprocessing
02-Jun-2025 10:10:41 Job status: running
02-Jun-2025 10:10:46 Job status: running
02-Jun-2025 10:10:51 Job status: running
02-Jun-2025 10:10:57 Job status: running
02-Jun-2025 10:11:02 Job status: running
02-Jun-2025 10:11:08 Job status: processing
02-Jun-2025 10:11:13 Job status: succeeded

02-Jun-2025 10:11:13 ✅ Done. Video generation succeeded.

SORA Generated video saved: 'videos/sora_02Jun2025_101029_Street_of_Tokyo__rainy_day_wit.mp4'
Done in 0 minu

In [32]:
Video(generated_video, width=860)

In [33]:
video_link = FileLink(path=generated_video)
video_link

/mnt/batch/tasks/shared/LS_root/mounts/clusters/seretkow8/code/Users/seretkow/SORA/videos/sora_02Jun2025_101029_Street_of_Tokyo__rainy_day_wit.mp4

In [34]:
!ls $OUTPUT_DIR -lh

total 75M
-rwxrwxrwx 1 root root 3.9M Jun  2 10:04 sora_02Jun2025_100343_A_young_boy_and_his_father_pla.mp4
-rwxrwxrwx 1 root root 1.1M Jun  2 10:04 sora_02Jun2025_100409_A_close_up_view_of_a_glass_sph.mp4
-rwxrwxrwx 1 root root 6.7M Jun  2 10:05 sora_02Jun2025_100427_Several_giant_wooly_mammoths_a.mp4
-rwxrwxrwx 1 root root  19M Jun  2 10:07 sora_02Jun2025_100505_Drone_view_of_waves_crashing_a.mp4
-rwxrwxrwx 1 root root 6.8M Jun  2 10:07 sora_02Jun2025_100711_Extreme_close_up_of_a_25_year_.mp4
-rwxrwxrwx 1 root root 6.8M Jun  2 10:08 sora_02Jun2025_100748_Extreme_close_up_of_a_50_year_.mp4
-rwxrwxrwx 1 root root  13M Jun  2 10:09 sora_02Jun2025_100826_Reflections_in_the_window_of_a.mp4
-rwxrwxrwx 1 root root 9.0M Jun  2 10:10 sora_02Jun2025_100940_A_drone_view_of_flock_of_paper.mp4
-rwxrwxrwx 1 root root 9.2M Jun  2 10:11 sora_02Jun2025_101029_Street_of_Tokyo__rainy_day_wit.mp4
